In [1]:
### Reflecting DB
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import func
from sqlalchemy import and_, or_, not_
import pandas as pd


In [2]:
# Create the engine to connect to a sqlite file.
engine = create_engine("sqlite:///CCES_Ver61.sqlite")

# Declare a base using automap_base()
Base = automap_base()


Base.prepare(engine, reflect=True)

# Print all of the classses mapped to the Base (this provides a list of the table names in the sqlite.db)
Base.classes.keys()

['Cces_16']

In [3]:
# Assign the CCES_16 class to a variable called 'CCES'
CCES = Base.classes.Cces_16

In [4]:
# Create a variable to manage session.queries later on...
session = Session(engine)
# session = Session(bind = engine)

# pd.query testing

In [65]:
stmt = session.query(CCES).statement
print(stmt)

SELECT "Cces_16".index, "Cces_16"."V101", "Cces_16".lookupzip, "Cces_16"."StateName", "Cces_16"."StateAbb", "Cces_16"."Latitude", "Cces_16"."Longitude", "Cces_16".countyfips, "Cces_16".birthyr, "Cces_16".gender, "Cces_16".educ, "Cces_16"."GunBackgroundChecks_16", "Cces_16"."ProhibitPublication_16", "Cces_16"."BanAssultWeapons_16", "Cces_16"."MakeCCPEasier_16", "Cces_16"."AlwaysAllowChoice_16", "Cces_16"."RapeIncestorHealth_16", "Cces_16"."ProhibitMoreThan20Weeks_16", "Cces_16"."Employersdeclinebenefits_16", "Cces_16"."ProhibitFedFunds_16", "Cces_16"."GayMarriage" 
FROM "Cces_16"


In [10]:
df = pd.read_sql_query(stmt, session.bind)
df.head(2)

,index,V101,lookupzip,StateName,StateAbb,Latitude,Longitude,countyfips,birthyr,gender,...,GunBackgroundChecks_16,ProhibitPublication_16,BanAssultWeapons_16,MakeCCPEasier_16,AlwaysAllowChoice_16,RapeIncestorHealth_16,ProhibitMoreThan20Weeks_16,Employersdeclinebenefits_16,ProhibitFedFunds_16,GayMarriage
0,0,222168628,3442,New Hampshire,NH,44.001231,-71.579923,33011.0,1969,female,...,Support,Oppose,Oppose,Oppose,Support,Oppose,Support,Oppose,Oppose,Support
1,1,301400441,3303,New Hampshire,NH,44.001231,-71.579923,33013.0,1948,female,...,Support,Oppose,Support,Oppose,Support,Oppose,Oppose,Oppose,Oppose,Support


In [15]:
print(list(df.columns)[11:])

['GunBackgroundChecks_16', 'ProhibitPublication_16', 'BanAssultWeapons_16', 'MakeCCPEasier_16', 'AlwaysAllowChoice_16', 'RapeIncestorHealth_16', 'ProhibitMoreThan20Weeks_16', 'Employersdeclinebenefits_16', 'ProhibitFedFunds_16', 'GayMarriage']


In [21]:
type(stmt)



sqlalchemy.sql.annotation.AnnotatedSelect

# List of dropdown variables

In [5]:
#############################################
# Dropdown Variables mapped to Sql...
#############################################
 
# Gun Questions
GunBackgroundChecks_16 = CCES.GunBackgroundChecks_16
ProhibitPublication_16 = CCES.ProhibitPublication_16
BanAssultWeapons_16 = CCES.BanAssultWeapons_16
MakeCCPEasier_16 = CCES.MakeCCPEasier_16

# Abortion Questions
AlwaysAllowChoice_16 = CCES.AlwaysAllowChoice_16
RapeIncestorHealth_16 = CCES.RapeIncestorHealth_16
ProhibitMoreThan20Weeks_16 = CCES.ProhibitMoreThan20Weeks_16
Employersdeclinebenefits_16 = CCES.Employersdeclinebenefits_16
ProhibitFedFunds_16 = CCES.ProhibitFedFunds_16

# Gay Marriage Question
GayMarriage = CCES.GayMarriage
#############################################

# Flask Route - metadata totals testing

In [ ]:
#############################################
# Creating metadata totals function for metadata totals flask route
#############################################

In [ ]:
#############################################
# Function to return Totals - Total Support and Total Oppose for each issue
#############################################
def sample_metadatatotals(sample):
    """Return the totals for a given sample"""
    TotalResults = []
    
    Approve = {}
    Oppose = {}
    
    ApproveTemp = session.query(sample).filter(sample == 'Support').count()
    Approve['Approve'] = ApproveTemp
    TotalResults.append(Approve)
    
    OpposeTemp = session.query(sample).filter(sample == 'Oppose').count()
    Oppose['Oppose'] = OpposeTemp
    TotalResults.append(Oppose)

    print(TotalResults)
#     return jsonify(TotalResults)

In [ ]:
#############################################
# Metadatatotals Function Testing 1

sample_metadatatotals(GayMarriage)

In [ ]:
#############################################
# Metadatatotals Function Testing 2

sample_metadatatotals(CCES.GunBackgroundChecks_16)

In [ ]:
#############################################
# Metadatatotals Function - without function - direct query to verify results

ApproveGay = session.query(CCES.GayMarriage).filter(CCES.GayMarriage == 'Support').count()
OpposeGay = session.query(CCES).filter(CCES.GayMarriage == 'Oppose').count()
print(f'Sqlite query of votes supporting Gay Marrige: {ApproveGay}')
print(f'Sqlite query of votes opposint Gay Marriage: {OpposeGay}')

# Flask Route - Metadata States Groupby Results

In [100]:
#############################################
# THIS ONE WON'T WORK HERE - IT IS MODIFIED TO WORK IN THE APP.PY
#############################################

def Metadata_States(sample):
    
    # Query to create two lists of tuples (state, number)
    #S1 [4] i
    StatesFor = db.session.query(Cces.StateName, Cces.Latitude, Cces.Longitude, func.count(Cces.CHANGE)).\
                filter(Cces.CHANGE == 'Support').\
                group_by(Cces.StateName).all()
    #S2 [1] m
    StatesForM = db.session.query(Cces.StateName, Cces.gender, func.count(Cces.CHANGE)).\
                filter(and_(Cces.CHANGE == 'Support'),(Cces.gender == 'male')).group_by(Cces.StateName).all()
    #S3 [1] f
    StatesForF = db.session.query(Cces.StateName, Cces.gender, func.count(Cces.CHANGE)).\
                filter(and_(Cces.CHANGE == 'Support'),(Cces.gender == 'female')).group_by(Cces.StateName).all()
    #O1 [1] j
    StatesNot = db.session.query(Cces.StateName, func.count(Cces.CHANGE)).\
                filter(Cces.CHANGE == 'Oppose').\
                group_by(Cces.StateName).all()
    #O2 [1] n
    StatesNotM = db.session.query(Cces.StateName, Cces.gender, func.count(Cces.CHANGE)).\
                filter(and_(Cces.CHANGE == 'Oppose'),(Cces.gender == 'male')).group_by(Cces.StateName).all()
    #O3 [1] g
    StatesNotF = db.session.query(Cces.StateName, Cces.gender, func.count(Cces.CHANGE)).\
                filter(and_(Cces.CHANGE == 'Oppose'),(Cces.gender == 'female')).group_by(Cces.StateName).all()

    # List that will hold final dictionaries - to be jsonified
    States_Results = []
    
    for i, m, f, j, n, g in zip(StatesFor, StatesForM, StatesForF, StatesNot, StatesNotM, StatesNotF):
        
        #Determine if the state is more in support or oppose
        if i[1] > j[1]:
            tempoverall = 'Support'
        else:
            tempoverall = 'Oppose'
        
        # Return Percentages rather than just raw data
        totalVotes = i[3] + j[1]

        # Build dictionary
        tempfile = {}
        tempfile['state'] = {
            'StateName' : i[0],
            'Lat' : i[1],
            'Long' : i[2]
        }
        tempfile['voteTotal'] = {
            'Support' : i[3],
            'Oppose' : j[1],
            'TotalVotes' : totalVotes,
            'Support_%' : float(format(i[3] / totalVotes, '.2f')),
            'Oppose_%' : float(format(j[1] / totalVotes, '.2f')),
            'Overall' : tempoverall
        }
        tempfile['genderVote'] = {
            'MaleSupport' : m[2],
            'FemaleSupport' : f[2],
            'MaleOppose' : n[2],
            'FemaleOppose' : g[2],
            'MaleTotal' : (m[2] + n[2]),
            'FemaleTotal' : (f[2] + g[2])
        }
        tempfile['genderVotePerc'] = {
            'MaleSupport' : float(format(m[2] / totalVotes, '.2f')),
            'FemaleSupport' : float(format(f[2] / totalVotes, '.2f')),
            'MaleOppose' : float(format(n[2] / totalVotes, '.2f')),
            'FemaleOppose' : float(format(g[2] / totalVotes, '.2f')),
        }
        States_Results.append(tempfile)

    return States_Results

In [6]:
#############################################
# Metadata States Groupby Function
#############################################

def Metadata_States(sample):
    
    # Query to create two lists of tuples (state, number)
    #S1 [4] i
    StatesFor = session.query(CCES.StateName, CCES.Latitude, CCES.Longitude, func.count(sample), CCES.StateAbb).\
                filter(sample == 'Support').\
                group_by(CCES.StateName).all()
    #S2 [1] m
    StatesForM = session.query(CCES.StateName, CCES.gender, func.count(sample)).\
                filter(and_(sample == 'Support'),(CCES.gender == 'male')).group_by(CCES.StateName).all()
    #S3 [1] f
    StatesForF = session.query(CCES.StateName, CCES.gender, func.count(sample)).\
                filter(and_(sample == 'Support'),(CCES.gender == 'female')).group_by(CCES.StateName).all()
    #O1 [1] j
    StatesNot = session.query(CCES.StateName, func.count(sample)).\
                filter(sample == 'Oppose').\
                group_by(CCES.StateName).all()
    #O2 [1] n
    StatesNotM = session.query(CCES.StateName, CCES.gender, func.count(sample)).\
                filter(and_(sample == 'Oppose'),(CCES.gender == 'male')).group_by(CCES.StateName).all()
    #O3 [1] g
    StatesNotF = session.query(CCES.StateName, CCES.gender, func.count(sample)).\
                filter(and_(sample == 'Oppose'),(CCES.gender == 'female')).group_by(CCES.StateName).all()

    # List that will hold final dictionaries - to be jsonified
    States_Results = []
    
    for i, m, f, j, n, g in zip(StatesFor, StatesForM, StatesForF, StatesNot, StatesNotM, StatesNotF):
        
        #Determine if the state is more in support or oppose
        if i[1] > j[1]:
            tempoverall = 'Support'
        else:
            tempoverall = 'Oppose'
        
        # Return Percentages rather than just raw data
        totalVotes = i[3] + j[1]

        # Build dictionary
        tempfile = {}
        tempfile['state'] = [
            {'StateName' : i[0]},
            {'Lat' : i[1]},
            {'Long' : i[2]},
            {'StateAbb' : i[4]}
                ]
        tempfile['voteTotal'] = [
            {'Support' : i[3]},
            {'Oppose' : j[1]},
            {'TotalVotes' : totalVotes},
            {'Support_%' : float(format(i[3] / totalVotes, '.2f'))},
            {'Oppose_%' : float(format(j[1] / totalVotes, '.2f'))},
            {'Overall' : tempoverall}
        ]
        tempfile['genderVote'] = [
            {'MaleSupport' : m[2]},
            {'FemaleSupport' : f[2]},
            {'MaleOppose' : n[2]},
            {'FemaleOppose' : g[2]},
            {'MaleTotal' : (m[2] + n[2])},
            {'FemaleTotal' : (f[2] + g[2])}
        ]
        tempfile['genderVotePerc'] = [
            {'MaleSupport' : float(format(m[2] / totalVotes, '.2f'))},
            {'FemaleSupport' : float(format(f[2] / totalVotes, '.2f'))},
            {'MaleOppose' : float(format(n[2] / totalVotes, '.2f'))},
            {'FemaleOppose' : float(format(g[2] / totalVotes, '.2f'))},
             ]
        States_Results.append(tempfile)

    return States_Results

In [7]:
xtemp = Metadata_States(GunBackgroundChecks_16)
xtemp

[{'state': [{'StateName': 'Alabama'},
   {'Lat': 32.601011199999995},
   {'Long': -86.6807365},
   {'StateAbb': 'AL'}],
  'voteTotal': [{'Support': 676},
   {'Oppose': 88},
   {'TotalVotes': 764},
   {'Support_%': 0.88},
   {'Oppose_%': 0.12},
   {'Overall': 'Oppose'}],
  'genderVote': [{'MaleSupport': 233},
   {'FemaleSupport': 443},
   {'MaleOppose': 57},
   {'FemaleOppose': 31},
   {'MaleTotal': 290},
   {'FemaleTotal': 474}],
  'genderVotePerc': [{'MaleSupport': 0.3},
   {'FemaleSupport': 0.58},
   {'MaleOppose': 0.07},
   {'FemaleOppose': 0.04}]},
 {'state': [{'StateName': 'Alaska'},
   {'Lat': 61.3025006},
   {'Long': -158.7750198},
   {'StateAbb': 'AK'}],
  'voteTotal': [{'Support': 85},
   {'Oppose': 24},
   {'TotalVotes': 109},
   {'Support_%': 0.78},
   {'Oppose_%': 0.22},
   {'Overall': 'Support'}],
  'genderVote': [{'MaleSupport': 40},
   {'FemaleSupport': 45},
   {'MaleOppose': 14},
   {'FemaleOppose': 10},
   {'MaleTotal': 54},
   {'FemaleTotal': 55}],
  'genderVotePerc':

In [8]:
testingLoop = xtemp[0]
testingLoop

{'state': [{'StateName': 'Alabama'},
  {'Lat': 32.601011199999995},
  {'Long': -86.6807365},
  {'StateAbb': 'AL'}],
 'voteTotal': [{'Support': 676},
  {'Oppose': 88},
  {'TotalVotes': 764},
  {'Support_%': 0.88},
  {'Oppose_%': 0.12},
  {'Overall': 'Oppose'}],
 'genderVote': [{'MaleSupport': 233},
  {'FemaleSupport': 443},
  {'MaleOppose': 57},
  {'FemaleOppose': 31},
  {'MaleTotal': 290},
  {'FemaleTotal': 474}],
 'genderVotePerc': [{'MaleSupport': 0.3},
  {'FemaleSupport': 0.58},
  {'MaleOppose': 0.07},
  {'FemaleOppose': 0.04}]}

In [10]:
print('this is object 1:  ', testingLoop['state'])
print('this is object, sublevel 0:  ',testingLoop['state'][0])
print('this is object, sublevel 1:  ',testingLoop['state'][1])
print('this is object, sublevel 2:  ',testingLoop['state'][2])
print('this is the state abb, sublevel 4  :', testingLoop['state'][3])
print('this is the state abb, sublevel 4  :', testingLoop['state'][3]['StateAbb'])
print(testingLoop['state'][2]['Long'])
latlong = [(testingLoop['state'][1]['Lat']), (testingLoop['state'][2]['Long'])]
for k in testingLoop['state'][0]:
    print('This is K in the loop:  ', k)
    print('this is after k in the loop:  ', testingLoop['state'][0]['StateName'])
print(latlong)
print(latlong[0])
print(' ')
print('Object 2:  ', testingLoop['voteTotal'])
print('Object 2:  ', testingLoop['voteTotal'][0])
print('Object 2:  ', testingLoop['voteTotal'][1])
print('Object 2:  ', testingLoop['voteTotal'][2])
print('Object 2:  ', testingLoop['voteTotal'][3])
print('Object 2:  ', testingLoop['voteTotal'][4])
print('Object 2:  ', testingLoop['voteTotal'][5])

this is object 1:   [{'StateName': 'Alabama'}, {'Lat': 32.601011199999995}, {'Long': -86.6807365}, {'StateAbb': 'AL'}]
this is object, sublevel 0:   {'StateName': 'Alabama'}
this is object, sublevel 1:   {'Lat': 32.601011199999995}
this is object, sublevel 2:   {'Long': -86.6807365}
this is the state abb, sublevel 4  : {'StateAbb': 'AL'}
this is the state abb, sublevel 4  : AL
-86.6807365
This is K in the loop:   StateName
this is after k in the loop:   Alabama
[32.601011199999995, -86.6807365]
32.601011199999995
 
Object 2:   [{'Support': 676}, {'Oppose': 88}, {'TotalVotes': 764}, {'Support_%': 0.88}, {'Oppose_%': 0.12}, {'Overall': 'Oppose'}]
Object 2:   {'Support': 676}
Object 2:   {'Oppose': 88}
Object 2:   {'TotalVotes': 764}
Object 2:   {'Support_%': 0.88}
Object 2:   {'Oppose_%': 0.12}
Object 2:   {'Overall': 'Oppose'}


In [10]:
TempList = []



testdict = {}
testdict['State'] = {
    'StateName' : 'MN',
    'Lat' : 123,
    'long' : 345
}
testdict['votetotal'] = {
    'support' : 514654,
    'oppose' : 1354563,
    'Overall' : 'support'
}

testdict['Gender'] = {
    'Msupport' : 54654,
    'Fsupport' : 54654,
    'Moppose' : 14545,
    'Foppose' : 665654
}
testdict

{'State': {'StateName': 'MN', 'Lat': 123, 'long': 345},
 'votetotal': {'support': 514654, 'oppose': 1354563, 'Overall': 'support'},
 'Gender': {'Msupport': 54654,
  'Fsupport': 54654,
  'Moppose': 14545,
  'Foppose': 665654}}

In [11]:
testdict['State']['Lat']

123

In [22]:
tempList = []

In [18]:
#############################################
# Metadata States Function Testing 1

Metadata_States(GunBackgroundChecks_16)

[{'State': 'Alabama',
  'Lat': 32.601011199999995,
  'long': -86.6807365,
  'Support': 676,
  'Oppose': 88,
  'Overall': 'Oppose'},
 {'State': 'Alaska',
  'Lat': 61.3025006,
  'long': -158.7750198,
  'Support': 85,
  'Oppose': 24,
  'Overall': 'Support'},
 {'State': 'Arizona',
  'Lat': 34.1682185,
  'long': -111.930907,
  'Support': 1265,
  'Oppose': 200,
  'Overall': 'Oppose'},
 {'State': 'Arkansas',
  'Lat': 34.7519275,
  'long': -92.13137840000002,
  'Support': 465,
  'Oppose': 60,
  'Overall': 'Oppose'},
 {'State': 'California',
  'Lat': 37.2718745,
  'long': -119.2704153,
  'Support': 5367,
  'Oppose': 493,
  'Overall': 'Oppose'},
 {'State': 'Colorado',
  'Lat': 38.9979339,
  'long': -105.55056699999999,
  'Support': 863,
  'Oppose': 121,
  'Overall': 'Oppose'},
 {'State': 'Connecticut',
  'Lat': 41.518783500000005,
  'long': -72.75750699999999,
  'Support': 660,
  'Oppose': 50,
  'Overall': 'Oppose'},
 {'State': 'Delaware',
  'Lat': 39.145251,
  'long': -75.41892059999999,
  'Sup

In [14]:
#############################################
# Filter by state how many respondents were supportive of Gun Background Checks
#############################################

StatesFor = session.query(CCES.StateName,CCES.Latitude, CCES.Longitude, func.count(CCES.GunBackgroundChecks_16)).filter(CCES.GunBackgroundChecks_16 == 'Support').group_by(CCES.StateName).all()
StatesNot = session.query(CCES.StateName, func.count(CCES.GunBackgroundChecks_16)).filter(CCES.GunBackgroundChecks_16 == 'Oppose').group_by(CCES.StateName).all()
print(StatesFor[0:5])
print(StatesNot[0:5])

[('Alabama', 32.601011199999995, -86.6807365, 676), ('Alaska', 61.3025006, -158.7750198, 85), ('Arizona', 34.1682185, -111.930907, 1265), ('Arkansas', 34.7519275, -92.13137840000002, 465), ('California', 37.2718745, -119.2704153, 5367)]
[('Alabama', 88), ('Alaska', 24), ('Arizona', 200), ('Arkansas', 60), ('California', 493)]


In [33]:
#############################################
# Groupby Function testing before building function
#############################################
StatesForAbortion = session.query(CCES.StateName, func.count(CCES.AlwaysAllowChoice_16)).filter(CCES.AlwaysAllowChoice_16 == 'Support').group_by(CCES.StateName).all()
StatesNotAbortion = session.query(CCES.StateName, func.count(CCES.AlwaysAllowChoice_16)).filter(CCES.AlwaysAllowChoice_16 == 'Oppose').group_by(CCES.StateName).all()

ThirdTry = []

for i, j in zip(StatesForAbortion, StatesNotAbortion):

    tempfile = {}
    tempfile['State'] = i[0]
    tempfile['Support'] = i[1]
    tempfile['Oppose'] = j[1]
    if i[1] > j[1]:
        tempoverall = 'Support'
    else:
        tempoverall = 'Oppose'
    tempfile['Overall'] = tempoverall
        
    ThirdTry.append(tempfile)
ThirdTry

[{'State': {'support': 'Alabama'},
  'Support': 397,
  'Oppose': 367,
  'Overall': 'Support'},
 {'State': {'support': 'Alaska'},
  'Support': 58,
  'Oppose': 51,
  'Overall': 'Support'},
 {'State': {'support': 'Arizona'},
  'Support': 867,
  'Oppose': 598,
  'Overall': 'Support'},
 {'State': {'support': 'Arkansas'},
  'Support': 231,
  'Oppose': 294,
  'Overall': 'Oppose'},
 {'State': {'support': 'California'},
  'Support': 4078,
  'Oppose': 1782,
  'Overall': 'Support'},
 {'State': {'support': 'Colorado'},
  'Support': 619,
  'Oppose': 365,
  'Overall': 'Support'},
 {'State': {'support': 'Connecticut'},
  'Support': 493,
  'Oppose': 217,
  'Overall': 'Support'},
 {'State': {'support': 'Delaware'},
  'Support': 187,
  'Oppose': 74,
  'Overall': 'Support'},
 {'State': {'support': 'District of Columbia'},
  'Support': 152,
  'Oppose': 34,
  'Overall': 'Support'},
 {'State': {'support': 'Florida'},
  'Support': 3111,
  'Oppose': 1744,
  'Overall': 'Support'},
 {'State': {'support': 'Georg

In [ ]:
#############################################
# Testing Groupby by specific state
# Filter by state how many respondents were supportive of Gun Background Checks
#############################################
AlabamaTotalExample = session.query(CCES.StateName, CCES.GayMarriage).filter(CCES.StateName == 'California').count()
AlabamaForExample = session.query(CCES.StateName, CCES.GayMarriage).filter(CCES.StateName == 'California').filter(CCES.GayMarriage == 'Support').count()
AlabamaAgainstExample = session.query(CCES.StateName, CCES.GayMarriage).filter(CCES.StateName == 'California').filter(CCES.GayMarriage == 'Oppose').count()
print(AlabamaTotalExample)
print(AlabamaForExample)
print(AlabamaAgainstExample)